In [1]:

import pandas as pd
import numpy as np
import datetime

import seaborn as sns
#from shapely.geometry import shape
import utils
import importlib

#from scipy import stats


In [11]:
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\hildlars\\OneDrive - Universitetet i Oslo\\NordicMathCovid\\PanNordicMobility\\FinalFiles_commutingPaper\\Code\\utils.py'>

In [2]:
def read_SWEDEN_csv(SWE_file):
    SWE_data = pd.read_csv(SWE_file, parse_dates=["DateTimeFromUtc", "DateTimeToUtc"], delimiter=";")


    SWE_data.loc[SWE_data['DetectorId'] == '53VIP0008-1','DetectorId'] = '53VIP0008_A'
    SWE_data.loc[SWE_data['DetectorId'] == '53VIP0008-2', 'DetectorId'] = '53VIP0008_A'
    SWE_data.loc[SWE_data['DetectorId'] == '53VIP0008-3','DetectorId'] = '53VIP0008_B'
    SWE_data.loc[SWE_data['DetectorId'] == '53VIP0008-4', 'DetectorId'] = '53VIP0008_B'

    # Sum over sensor
    SWE_data = SWE_data[['DetectorId', 'DateTimeFromUtc', 'FlowVpm']].groupby(['DetectorId', 'DateTimeFromUtc']).sum().reset_index()
    
    
    # Date and time
    SWE_data['date'] = SWE_data['DateTimeFromUtc'].apply(lambda x: x.date())
    SWE_data['date'] = pd.to_datetime(SWE_data['date'])
    SWE_data['hour'] = SWE_data['DateTimeFromUtc'].apply(lambda x: x.hour)
    SWE_data['minute'] = SWE_data['DateTimeFromUtc'].apply(lambda x: x.minute)
    
    
    # Select the columns needed
    SWE_data = SWE_data[['DetectorId', 'date', 'hour','minute', 'FlowVpm']]


    # Merge origin and destination country with data
    SensorCountry = pd.DataFrame(
        [["53VIP0008_A", "DEN", 'SWE','1'], ['53VIP0008_B', 'SWE', 'DEN', '2']], columns=["DetectorId", "origin_country", "dest_country", 'sensor_dir'])
    SWE_data = pd.merge(SWE_data, SensorCountry, on='DetectorId')
    SWE_data.loc[SWE_data['DetectorId'] == '53VIP0008_A','DetectorId'] = '53VIP0008'
    SWE_data.loc[SWE_data['DetectorId'] == '53VIP0008_B', 'DetectorId'] = '53VIP0008'

    # Rename and select columns
    SWE_data = SWE_data.rename(columns={'DetectorId': 'sensor_id', 'FlowVpm': 'total_vehicles'})
    return(SWE_data)

Read data from .csv files

In [3]:
SWE_file = "../Data/DenSwe/Öresundsbron_2022-04-21_2.csv"
data = read_SWEDEN_csv(SWE_file)

In [4]:
def agg_data_DENSWE(data,min_date = datetime.datetime(2019, 1, 1)  , max_date = datetime.datetime(2021, 12, 31)):

    agg_data = data.copy()

    ##Create a new variables that contain both sensor name and country name
    agg_data["sensor_id"] = agg_data["sensor_id"].astype(str)
    agg_data['sensor_origin'] = agg_data[['sensor_id', 'origin_country']].agg(', '.join, axis=1)
    agg_data['sensor_destination'] = agg_data[['sensor_id', 'dest_country']].agg(', '.join, axis=1)


    agg_data["minute"] = pd.to_timedelta(agg_data["minute"], unit="min")
    agg_data["hour"] = pd.to_timedelta(agg_data["hour"], unit="h")


    agg_data['date'] = agg_data['date'] + agg_data['hour'] + agg_data['minute']
    agg_data = agg_data[(agg_data.date > min_date) & (agg_data.date < max_date)].copy().reset_index()
    

    min_date = agg_data.date.min()
    max_date = agg_data.date.max()

    agg_data = agg_data.groupby(['sensor_origin', 'sensor_destination', 'date'])['total_vehicles'].sum().reset_index().copy()
    rm_idx = np.where(np.isnan(agg_data.total_vehicles))
    agg_data = agg_data.drop(index = rm_idx[0])
    agg_data['total_vehicles'] = agg_data['total_vehicles'].apply(lambda x: int(x))

    f = lambda x: x.reindex(pd.date_range(min_date,
                                            max_date,
                                            name='date',
                                            freq='1min'), fill_value=0)


    agg_data = (agg_data.set_index('date')
                .groupby(["sensor_origin", "sensor_destination"])["total_vehicles"]
                .apply(f)
                .reset_index())
    agg_data = agg_data.pivot_table(index=["sensor_origin","sensor_destination"], columns=["date"],values=["total_vehicles"] ).droplevel(level = 0,axis = 1 )

    return(agg_data)

In [5]:

min_date = datetime.datetime(2019, 1, 1)
#max_date = datetime.datetime(2021, 12, 31)
max_date = datetime.datetime.combine(date = data['date'].max().date(), time = datetime.time())

In [6]:
# For 2017-2023 this takes about 4 min to run
agg_data = agg_data_DENSWE(data, min_date = min_date  , max_date = max_date)

In [7]:
# This takes about 3 min to run
models_pre, data_pre = utils.fit_period(agg_data, d1=datetime.date(2019,1,1), d2=datetime.date(2020,3,1), hourly = False, nSamp = 10000, Normalize = False, N = 10, seed=2024, FitMethod = 'Bayesian')



c:\Users\hildlars\OneDrive - Universitetet i Oslo\NordicMathCovid\PanNordicMobility\FinalFiles_commutingPaper\Code\utils.py:122: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  models = data[c].reset_index("weekday").groupby("weekday").agg(lambda x : fit_model(x,hourly_data, nSamp, N = N, hourminute = True,seed = seed, FitMethod = FitMethod))


In [8]:

# Save the fitted model and data
models_pre.to_pickle('../Data/models_den.pkl')
data_pre.to_pickle('../Data/data_den.pkl')
agg_data.to_pickle('../Data/agg_data_den.pkl')